In [1]:
from sympy import *

In [2]:
def multiply(m, j0, j1, a, b, c, d):
    for i in range(m.rows):
        x, y = m[i, j0], m[i, j1]
        m[i, j0] = a * x + c * y
        m[i, j1] = b * x + d * y

In [3]:
def smith(m, domain=ZZ):
    m = Matrix(m)
    s = eye(m.rows)
    t = eye(m.cols)
    last_j = -1
    for i in range(m.rows):
        for j in range(last_j+1, m.cols):
            if not m.col(j).is_zero:
                break
        else:
            break
        if m[i,j] == 0:
            for ii in range(m.rows):
                if m[ii,j] != 0:
                    break
            for counter in range(m.cols):
                x, y = m[i, counter], m[ii, counter]
                m[i, counter] = y
                m[ii, counter] = x
            multiply(s, i, ii, 0, 1, 1, 0)
        multiply(m, j, i, 0, 1, 1, 0)
        for counter in range(t.cols):
            x, y = t[j, counter], t[i, counter]
            t[j, counter] = y
            t[i, counter] = x
        j = i
        upd = True
        while upd:
            upd = False
            for ii in range(i+1, m.rows):
                if m[ii, j] == 0:
                    continue
                upd = True
                if domain.rem(m[ii, j], m[i, j]) != 0:
                    coefficient_1, coefficient_2, g = domain.gcdex(m[i,j], m[ii, j])
                    coefficient_3 = domain.quo(m[ii, j], g)
                    coefficient_4 = domain.quo(m[i, j], g)
                    for counter in range(m.cols):
                        x, y = m[i, counter], m[ii, counter]
                        m[i, counter] = coefficient_1 * x + coefficient_2 * y
                        m[ii, counter] = -coefficient_3 * x + coefficient_4 * y
                    multiply(s, i, ii, coefficient_4, -coefficient_2, coefficient_3, coefficient_1)
                coefficient_5 = domain.quo(m[ii, j], m[i, j])
                for counter in range(m.cols):
                    x, y = m[i, counter], m[ii, counter]
                    m[i, counter] = 1 * x + 0 * y
                    m[ii, counter] = -coefficient_5 * x + 1 * y
                multiply(s, i, ii, 1, 0, coefficient_5, 1)
            for jj in range(j+1, m.cols):
                if m[i, jj] == 0:
                    continue
                upd = True
                if domain.rem(m[i, jj], m[i, j]) != 0:
                    coefficient_1, coefficient_2, g = domain.gcdex(m[i,j], m[i, jj])
                    coefficient_3 = domain.quo(m[i, jj], g)
                    coefficient_4 = domain.quo(m[i, j], g)
                    multiply(m, j, jj, coefficient_1, -coefficient_3, coefficient_2, coefficient_4)
                    for counter in range(t.cols):
                        x, y = t[j, counter], t[jj, counter]
                        t[j, counter] = coefficient_4 * x + coefficient_3 * y
                        t[jj, counter] = -coefficient_2 * x + coefficient_1 * y
                coefficient_5 = domain.quo(m[i, jj], m[i, j])
                multiply(m, j, jj, 1, -coefficient_5, 0, 1)
                for counter in range(t.cols):
                    x, y = t[j, counter], t[jj, counter]
                    t[j, counter] = 1 * x + coefficient_5 * y
                    t[jj, counter] = 0 * x + 1 * y
        last_j = j
    for i1 in range(min(m.rows, m.cols)):
        for i0 in reversed(range(i1)):
            coefficient_1, coefficient_2, g = domain.gcdex(m[i0, i0], m[i1,i1])
            if g == 0:
                continue
            coefficient_3 = domain.quo(m[i1, i1], g)
            coefficient_4 = domain.quo(m[i0, i0], g)
            for counter in range(m.cols):
                x, y = m[i0, counter], m[i1, counter]
                m[i0, counter] = 1 * x + coefficient_2 * y
                m[i1, counter] = coefficient_3 * x + (coefficient_2 * coefficient_3 - 1) * y
            multiply(s, i0, i1, 1-coefficient_2*coefficient_3, coefficient_2, coefficient_3, -1)
            multiply(m, i0, i1, coefficient_1, 1-coefficient_1*coefficient_4, 1, -coefficient_4)
            for counter in range(t.cols):
                x, y = t[i0, counter], t[i1, counter]
                t[i0, counter] = coefficient_4 * x + (1 - coefficient_1 * coefficient_4) * y
                t[i1, counter] = x + -coefficient_1 * y
    return (s, m, t)